In [16]:
# import libs
import boto3
import json

In [17]:
client = boto3.client("lambda")

s3 = boto3.client("s3")

In [18]:
# functions

def get_object_body(bucket: str, key: str) -> bytes:
    response = s3.get_object(Bucket=bucket, Key=key)
    return response["Body"].read().decode("utf-8")


def invoke_lambda(function_type:str,before_id:str,after_id:str,before_page:str,after_page:str):
    payload = {
        "body": {
            "bucket_name": "aska-image-bucket-dev",
            "before": {"json_object_key":f"{before_id}/{before_page}.json","image_object_key":f"{before_id}/{before_page}.png"},
            "after": {"json_object_key":f"{after_id}/{after_page}.json","image_object_key":f"{after_id}/{after_page}.png"},
            "params" : {
                "match_threshold" : 0.85,
                "threshold" : 220,
                "eps" : 20,
                "min_samples" : 50,
            },
        }
    }

    response = client.invoke(
        FunctionName=function_type,
        InvocationType="RequestResponse",
        LogType="Tail",
        Payload=bytes(json.dumps(payload).encode()),
    )
    print(response["Payload"].read().decode("utf-8"))


In [22]:
# main
if __name__ == '__main__':
    invoke_lambda("aska-api-dev-ImageDiffHandler","id231321","id231321","1","2")

{"statusCode": 200, "body": "{\"message\": \"Created and sdaved diff image\"}"}
